In [4]:
TRAIN_PATH = "CovidDatset/train"
VAL_PATH = "CovidDatset/val"

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing import *
from tensorflow.keras.layers.experimental import preprocessing

# Load Training and Validation Set

In [6]:
#TRAIN SET

def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    return image, label


ds_train_ = image_dataset_from_directory(
    "CovidDatset/train",
    labels = "inferred",
    label_mode = "binary",
    image_size = [224, 224],
    batch_size = 32,
    shuffle = True,
)


AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

Found 312 files belonging to 2 classes.


In [7]:
#VALIDATION SET

ds_valid_ = image_dataset_from_directory(
    "CovidDatset/val",
    labels = "inferred",
    label_mode = "binary",
    image_size = [224, 224],
    batch_size = 32,
    shuffle = True,
)

AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

Found 80 files belonging to 2 classes.


# CREATING THE CNN MODEL

In [8]:
model = keras.Sequential([
    
    layers.InputLayer(input_shape=[224, 224, 3]),
    
   #DATA AUGMENTATION
    preprocessing.Rescaling(1./255),
    preprocessing.RandomFlip(mode='horizontal'),

    # Block One
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Dropout(0.2),

    # Block Two
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Dropout(0.2),

    # Block Three
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Dropout(0.2),

    # Head
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
      
])

# COMPILING THE MODEL

In [13]:
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ['binary_accuracy'],
)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    min_delta = 0.001,
    patience = 20,
    restore_best_weights = True,
)

history = model.fit(
    ds_train,
    validation_data = ds_valid,
    epochs = 40,
    callbacks = [early_stopping],
    verbose = 0,
)